In [1]:
from __future__ import print_function # so print doesn't show brackets
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
import pickle
pickle.HIGHEST_PROTOCOL = 2
sys.path.append(os.path.join("..", "Libraries","QML_lib"))
from RedisSettings import *
import Evo as evo
import DataBase 
from QMD import QMD #  class moved to QMD in Library
import QML
import ModelGeneration 
import BayesF
import matplotlib.pyplot as plt
%matplotlib inline
from pympler import asizeof
import matplotlib.pyplot as plt
paulis = ['x', 'y', 'z'] # will be chosen at random. or uncomment below and comment within loop to hard-set

import time as time 
import argparse
parser = argparse.ArgumentParser(description='Pass variables for (I)QLE.')



/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [7]:
num_runs=1
num_particles=300
num_experiments = 50
num_times_bayes = 'all'
qle=True

#initial_op_list = ['x', 'y', 'z', 'xTx', 'yTz', 'xTz']
initial_op_list = ['x', 'y']
#true_op = random.choice(initial_op_list)
#true_params = [np.random.rand() for i in range(n_pars)]



for i in range(num_runs):
#    true_op=initial_op_list[i%2]
    true_op= 'x'
    true_params = [np.random.rand()]
    qmd = QMD(
        initial_op_list=initial_op_list, 
        true_operator=true_op, 
        true_param_list=true_params, 
        num_particles=num_particles,
        num_experiments = num_experiments, 
        num_times_for_bayes_updates = num_times_bayes,
        qle=qle,
        num_probes=5,
        max_num_branches = 0,
        max_num_qubits = 2, 
        parallel = True
    )
   # qmd.learnModelNameList(model_name_list=['z'], blocking=True, use_rq=False)
    qmd.blockingQMD()


Running  QLE  for true operator  x  with parameters :  [0.9977111027916642]
Model  x  not previously considered -- adding.
Initialization Ready
Model  y  not previously considered -- adding.
Initialization Ready
QHL for x
Initialization Ready
Initial time selected > 0.779546032058
Final time selected > 223.658069888
Final Parameters mean and stdev:[ 0.97467332  0.0349326 ]
QHL for y
Initialization Ready
Initial time selected > 0.613384944516
Final time selected > 12.9744059188
Final Parameters mean and stdev:[ 0.98745209  0.02574767]
Loglikelihoods for x
Before: -26.9977425392
After: -47.6475877562
Sum of data: 18  1s out of  50 = 36.0 %
Loglikelihoods for y
Before: -38.7375906898
After: -76.027381452
Sum of data: 13  1s out of  50 = 26.0 %
Bayes factor bw x / y = 2.11440316214e+12
Bayes Correct
Bayes already computed bw 1.0 0.0


In [ ]:
from QML import modelClassForRemoteBayesFactor

In [ ]:
z=modelClassForRemoteBayesFactor(0)
x=modelClassForRemoteBayesFactor(1)

In [ ]:
z.Updater.normalization_record

In [ ]:
x.Updater.normalization_record


In [ ]:
x.Updater.log_total_likelihood

In [ ]:
qmd.ProbeDict[1,1]

In [ ]:
def get_exp(model, time):
    gen = model.Updater.model # or gen=model.GenSimModel
    exp = np.empty(len(time), dtype=gen.expparams_dtype)
    exp['t'] = time

    for i in range(1, len(gen.expparams_dtype)):
        col_name = 'w_'+str(i)
        exp[col_name] = model.FinalParams[i-1,0] 
    return exp


In [ ]:
from QML import modelClassForRemoteBayesFactor
mod=modelClassForRemoteBayesFactor(0)
times=mod.Times
min_time = min(mod.Times)
max_time  = max(mod.Times)
times = np.arange(min_time, max_time, 1)
exps= [get_exp(mod, [time]) for time in times]
true_params=qmd.TrueParamsList

In [ ]:
sim_times=mod.Updater.model.simulate_experiment(np.array([true_params]), np.array(exps))
sim_data = sim_times[0,0]
analytical_times_1 =  np.cos(true_params*times)**2

In [ ]:
def analytical_expec_value(param, time):
    return (np.cos(param*time))**2
    

In [ ]:
analytical_times = []
for t in times:
    analytical_times.append(analytical_expec_value(true_params[0], t))

In [ ]:
np.all(analytical_times == analytical_times_1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(times, analytical_times)
plt.scatter(times, sim_data)


plt.xlim(0, 50)

plt.show()

In [ ]:
gen = mod.GenSimModel
modelparams = np.array([true_params])
expparams = exps[0]
all_outcomes = gen.domain(expparams)

In [ ]:
gen.likelihood(all_outcomes, modelparams, expparams)

In [ ]:
expparams

In [ ]:
liks = []
for exp in exps:
    all_outcomes = gen.domain(exp)
    liks.append(gen.likelihood(all_outcomes, modelparams, exp))

In [ ]:
len(liks)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(times, analytical_times)
plt.scatter(times, liks)


plt.xlim(0, 50)

plt.show()

In [ ]:
from Evo import expectation_value
import DataBase as db

In [ ]:
true_params

In [ ]:
trueop = 'z'
ham = db.empty_array_of_same_dim(trueop)
op = db.operator(trueop)
constituents = op.constituents_operators
ideal_probe = op.ideal_probe


for i in range(len(true_params)):
    ham += true_params[i] * constituents[i]
    
    

In [ ]:
expec_values = []
for t in times:
    expec_values.append(expectation_value(ham, t, ideal_probe))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(times, analytical_times)
plt.scatter(times, expec_values)


plt.xlim(0, 50)

plt.show()

In [ ]:
old_mod._trueparams

In [ ]:
new_mod._trueparams

In [ ]:
for att in dir(old_mod):
    equal_check=True
    try:
        if old_mod.__getattribute__(att) != new_mod.__getattribute__(att) :
            equal_check=False
        run_check = True
    except ValueError:
        run_check = False
        continue
    
    if run_check:
        if not equal_check:
            print("Attributes unequal:", att)

In [ ]:
old_mod.base_model.simulate_experiment()

In [ ]:
for att in dir(old):
    equal_check=True
    try:
        if old.__getattribute__(att) != new.__getattribute__(att) :
            equal_check=False
        run_check = True
    except ValueError:
        run_check = False
        continue
    except AttributeError:
        run_check = False
        continue
    
    if run_check:
        if not equal_check:
            print("Attributes unequal:", att)

In [ ]:
old._data_record == new.data_record

In [ ]:
new.plot_posterior_marginal()

In [ ]:
old.plot_posterior_marginal()

In [ ]:
print(old.est_mean())
print(new.est_mean())

In [ ]:
new.plot_posterior_marginal()

In [ ]:
old.plot_posterior_marginal()

In [ ]:
x.FinalParams

In [ ]:
dir(new)

In [ ]:
new.log_total_likelihood

In [ ]:
old.log_total_likelihood

In [ ]:
asizeof.asizeof(pickle.dumps(new))

In [ ]:
def analytical_epxec_value(t, param):
    # valid only for H=x,y I think
    return (np.cos(param*t/2))**2 
#+ (np.sin(param*t/2))
    

In [ ]:
for t in np.arange(0, 3.14, 0.1):
    print(analytical_epxec_value(t, param=2))

In [ ]:
analytical_epxec_value(t=1.67, param=0.4)

In [ ]:
x.FinalParams

In [ ]:
x_db = pickle.loads(learned_models_info.get('0.0'))

In [ ]:
pr=x_db['final_prior']

In [ ]:
pr.sample()